In [1]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


#Loading google maps api with query "Food trucks in Indianapolis"

In [50]:
import requests

# Define the API endpoint and parameters
endpoint = "https://maps.googleapis.com/maps/api/place/textsearch/json"
params = {
    'query': 'food trucks in Indianapolis',
    'key': '###'
}

# Make the request to the API
response = requests.get(endpoint, params=params)

# Parse the response JSON
data = response.json()

In [52]:
#Saving json so we dont need to retrieve it everytime
import json
with open('data.json', 'w') as file:

    json.dump(data, file, indent=4)

In [28]:
import pandas as pd

# Extract the relevant fields from the data
food_trucks = []
for place in data['results']:
    food_trucks.append({
        'name': place['name'],
        'address': place['formatted_address'],
        'rating': place.get('rating', None),
        'website': place.get('website', None),
        'open_hour': place.get('opening_hours', {}).get('open_now', None),
        'cuisine_type': place.get('types', [])
    })

# Convert the data into a pandas DataFrame
df = pd.DataFrame(food_trucks)

In [55]:
import requests


place_ids = [place for place in df.place_id]


endpoint = "https://maps.googleapis.com/maps/api/place/details/json"


place_details_list = []


for place_id in place_ids:

    params = {
        'place_id': place_id,
        'key': '###'
    }


    response = requests.get(endpoint, params=params)

    results_data = response.json()


    place_details = results_data['result']
    place_details_list.append(place_details)


In [112]:
import re
def get_times(x):
  if x==None:
    return None
  days_times = x

  # Create lists to store the opening and closing times
  open_times = []

  # Process each item in the list
  for item in days_times:
      # Extract the day and times
      day, times = item.split(': ', 1)

      if times != 'Closed':

          cleaned_times = re.sub(r'[^\x00-\x7F-:]+', '', times)

          # open_time, close_time = cleaned_times[:len(cleaned_times) // 2],cleaned_times[len(cleaned_times) // 2:]


          open_times.append( re.sub(r'(\d{1,2}:\d{2})([APMapm]{2})(\d{1,2}:\d{2})([APMapm]{2})', r'\1\2 - \3\4', cleaned_times)
)




  output = f"{open_times[0]}"

  return output

In [149]:
additional_details = []
for place in place_details_list:

    additional_details.append({
        'place_id': place.get('place_id', None) ,
        'Cuisine': place.get('types', None) ,
        'website': place.get('website', None) ,
        'Hours': get_times(place.get('current_opening_hours',{}).get('weekday_text',None))

    })



In [150]:
df2 = pd.DataFrame(additional_details)

In [130]:
merged_df = pd.merge(df, df2, on='place_id')

In [139]:
print(merged_df.head())

                      place_id                           name  \
0  ChIJrZbSAjlda4gRzX8ft2gUhfw      big Jerry Indy food truck   
1  ChIJ5yom11tWa4gR1fqgVmHn10o       Pico de Gallo Food Truck   
2  ChIJ1WvY2R5Ja4gRoI-kU1qn1Bc  The Latin Flavor - Food Truck   
3  ChIJG__HbklVa4gRYiCltEzZ5ps  Mi Lindo Pénjamo - Food Truck   
4  ChIJA1KNvZVPa4gRzYcJrty8Ews         Super Tacos food truck   

                                             address  rating  \
0  8522 Madison Ave, Indianapolis, IN 46227, Unit...     5.0   
1  3014 Lafayette Rd #3000, Indianapolis, IN 4622...     4.5   
2  9130 E 38th St, Indianapolis, IN 46235, United...     4.1   
3  7910 Michigan Rd, Indianapolis, IN 46268, Unit...     4.5   
4  7202 E Washington St, Indianapolis, IN 46219, ...     4.6   

                                             Cuisine  \
0  [restaurant, food, point_of_interest, establis...   
1  [restaurant, food, point_of_interest, establis...   
2  [restaurant, food, point_of_interest, establis...   


In [143]:
merged_df=merged_df.drop('place_id',axis=1)

In [148]:
merged_df.to_csv('Generated_data.csv', index=False)

#Merged df contains the generated data with the required info, there has been cleaning operation performed on the Hours attribute. The google places api does not return the exact type of cuisine the restaurant offers. We can use the generated data to not chart out a plan for the food trip
#The factors that are selected for choosing a food truck are opening times, location and the ratings